# Bot Testing Grounds
This notebook tests the performance of a series of bots from draftsimtools.

### Package importing and parameter setting

In [1]:
# Imports packages 

import warnings
warnings.filterwarnings('ignore')

import pickle
import ast
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.dataset import Dataset

# Workaround for variable Jupyter directories
import sys
sys.path.append('bots')

import draftsimtools as ds
from draftsimtools import DraftNet

In [2]:
# Sets pytorch device
device = torch.device("cpu") 

# Sets file paths
jsonPath = "../../data/AllSets.json"
ratingPath = "../../data/standardized_m19/standardized_m19_rating.tsv"
draftPath = "../../data/standardized_m19/drafts_test.pkl"

# Sets file paths for Bayesian bot
pCollPath = "bots_data/bayes_pCoDraft.csv"
pPackPath = "bots_data/bayes_pChoice.csv"
pFullPath = "bots_data/bayes_pFull.csv"
namesPath = "bots_data/bayes_names.csv"

### Reads in data
Data folder is downloadable from draft-data-files on slack. 

In [3]:
# Loads drafts
drafts = None
with open(draftPath, "rb") as f:
    drafts = pickle.load(f)

# Loads ratings
m19_set = pd.read_csv(ratingPath, delimiter="\t", converters={6:ast.literal_eval})

# Label-encodes card names
le = ds.create_le(m19_set["Name"].values)

In [7]:
# Subsets drafts for faster runtimes - for real testing, use all drafts
subset_drafts = drafts[:500] 

In [5]:
# display(m19_set)
# for col in m19_set.columns: 
#     print(col) 

### Creates bots

In [5]:
# Instantiates heuristic-based bots
bot1 = ds.RandomBot() 
bot2 = ds.RaredraftBot(m19_set) 
bot3 = ds.ClassicBot(m19_set) 
bot4 = ds.BayesBot(le, pCollPath, pPackPath, pFullPath, namesPath)

# Loads neural net from saved pytorch file
test_net = torch.load("bots_data/draftnet_oct_17_2019_633_cpu.pt")
test_net.eval()

# Instantiates neural-network bot
bot5 = ds.NeuralNetBot(test_net, le)

Name
0            Abnormal_Endurance
1                Act_of_Treason
2          Aegis_of_the_Heavens
3               Aerial_Engineer
4                 Aether_Tunnel
5        Aethershield_Artificer
6            Ajani's_Last_Stand
7             Ajani's_Pridemate
8               Ajani's_Welcome
9    Ajani_Adversary_of_Tyrants
10                  Alpine_Moon
11        Amulet_of_Safekeeping
12            Angel_of_the_Dawn
13                   Anticipate
14                Apex_of_Power
15       Arcades_the_Strategist
16          Arcane_Encyclopedia
17               Aven_Wind_Mage
18             Aviation_Pioneer
19                     Banefire
20             Blanchwood_Armor
21             Blood_Divination
22                Boggart_Brute
23                   Bogstomper
24                  Bone_Dragon
25                  Bone_to_Ash
26              Brawl-Bash_Ogre
27               Bristling_Boar
28                       Cancel
29           Catalyst_Elemental
..                          ...
235

In [8]:
# Tests all bots in the testing framework
tester = ds.BotTester(subset_drafts)
#tester.evaluate_bots([bot1, bot2, bot3, bot4, bot5], ["RandomBot", "RaredraftBot", "ClassicBot", "BayesBot", "NNetBot"])
tester.evaluate_bots([bot1, bot2, bot3, bot4], ["RandomBot", "RaredraftBot", "ClassicBot", "BayesBot"])
tester.report_evaluations()
tester.write_evaluations()

draft_num       250.500000
pick_num         23.000000
RandomBot         0.219600
RaredraftBot      0.429556
ClassicBot        0.450000
BayesBot          0.142622
dtype: float64
Wrote correct to: exact_correct.tsv
Wrote fuzzy_correct to: fuzzy_correct.tsv
Wrote rank_error to: rank_error.tsv
Wrote card_acc to: card_accuracies.tsv


### Load pre-trained neural network model

### Runs on an example pack

In [11]:
# Instantiates bot tester
tester = ds.BotTester(subset_drafts)

# Create demo collection
demo_collection = tester.drafts[0][0]
demo_pack = tester.drafts[0][1]
demo_x = ds.collection_pack_to_x(demo_collection, demo_pack, le)

# Return the result
result = test_net(demo_x)

# Maps numeric classes to card names and displays result
pack_dict = {str(le.inverse_transform([i])[0]) : float(v.detach().numpy()) for i, v in enumerate(result[0,:]) if v > 0}
display(pack_dict)

{'Anticipate': 12.157964706420898,
 'Cancel': 11.781457901000977,
 'Crash_Through': 10.274678230285645,
 'Daggerback_Basilisk': 13.520963668823242,
 'Dwindle': 14.012452125549316,
 'Epicure_of_Blood': 12.799198150634766,
 'Exclusion_Mage': 16.070850372314453,
 'Hired_Blade': 11.71932315826416,
 'Hostile_Minotaur': 11.677688598632812,
 "Knight's_Pledge": 13.288070678710938,
 'Knightly_Valor': 17.361173629760742,
 'Plains_4': 8.91723918914795,
 'Rise_from_the_Grave': 13.873148918151855,
 'Skeleton_Archer': 12.811566352844238}

## Testing all bots against each other